# Folder Processing - Batch 3D Centroid Extraction

This notebook demonstrates how to process multiple BlastoSpim NPZ files in a folder to extract geometric centers (centroids) from 3D volume labels using the correct voxel spacing from the research paper.

## Overview
- **Dataset**: BlastoSpim 3D microscopy data
- **Voxel Spacing**: Z=2.0µm, Y/X=0.208µm (from research paper)
- **Method**: Batch processing with center of mass calculation
- **Output**: Organized results with comprehensive visualizations and analysis

## Features
- ✅ Batch processing of entire directories
- ✅ Comprehensive logging for all files
- ✅ Detailed progress tracking with statistics
- ✅ Robust error handling per file
- ✅ Research paper compliant voxel spacing
- ✅ Professional visualizations for each file
- ✅ Organized output structure
- ✅ Batch summary reports
- ✅ Parallel processing capabilities

## 1. Import Required Libraries

Import all necessary libraries for batch processing, including file handling, progress tracking, and parallel processing capabilities.

In [1]:
# Standard library imports for file and folder operations
import sys
import glob
import logging
from pathlib import Path
from datetime import datetime
import time

# Scientific computing and progress tracking
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add project root to Python path
project_root = '/mnt/home/dchhantyal/centroid_model_blastospim'
if project_root not in sys.path:
    sys.path.append(project_root)

# Import our custom batch processing modules
from src.preprocessing.centroid_batch_processor import CentroidBatchProcessor
from src.utils.config import ConfigManager

print("✅ All libraries imported successfully!")
print(f"✅ Project root: {project_root}")
print(f"✅ Python version: {sys.version}")
print(f"✅ Key libraries:")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")
print(f"   - Matplotlib: {plt.matplotlib.__version__}")

✅ All libraries imported successfully!
✅ Project root: /mnt/home/dchhantyal/centroid_model_blastospim
✅ Python version: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:27:50) [GCC 13.3.0]
✅ Key libraries:
   - NumPy: 2.3.0
   - Pandas: 2.3.0
   - Matplotlib: 3.10.3


## 2. Setup Comprehensive Logging

Configure detailed logging for batch processing with separate logs for overall batch operations and individual file processing.

In [2]:
# Create logs directory for batch processing
logs_dir = Path("notebook_logs/batch_processing")
logs_dir.mkdir(parents=True, exist_ok=True)

# Configure comprehensive logging for batch operations
batch_log_filename = logs_dir / f"batch_processing_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

# Create batch logger
batch_logger = logging.getLogger('batch_processor')
batch_logger.setLevel(logging.DEBUG)

# Clear any existing handlers
batch_logger.handlers.clear()

# Create formatters for different log levels
detailed_formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s'
)
console_formatter = logging.Formatter(
    '%(levelname)s: %(message)s'
)

# File handler for detailed batch logs
batch_file_handler = logging.FileHandler(batch_log_filename)
batch_file_handler.setLevel(logging.DEBUG)
batch_file_handler.setFormatter(detailed_formatter)

# Console handler for progress information
batch_console_handler = logging.StreamHandler()
batch_console_handler.setLevel(logging.INFO)
batch_console_handler.setFormatter(console_formatter)

# Add handlers to batch logger
batch_logger.addHandler(batch_file_handler)
batch_logger.addHandler(batch_console_handler)

# Test logging system
batch_logger.info("🚀 Batch processing logging system initialized")
batch_logger.info(f"📁 Batch log file: {batch_log_filename}")
batch_logger.debug("Debug logging is working for batch operations")

print(f"✅ Batch logging configured successfully")
print(f"📋 Batch log file: {batch_log_filename}")
print(f"📊 Log level: {batch_logger.level} (DEBUG)")
print(f"🔧 Handlers: {len(batch_logger.handlers)} (file + console)")
print(f"📂 Log directory: {logs_dir}")

# Additional logging for individual file processing
individual_logs_dir = logs_dir / "individual_files"
individual_logs_dir.mkdir(exist_ok=True)

print(f"📁 Individual file logs will be saved to: {individual_logs_dir}")
print("📝 This setup allows tracking both batch overview and individual file details")

INFO: 🚀 Batch processing logging system initialized
INFO: 📁 Batch log file: notebook_logs/batch_processing/batch_processing_20250619_154702.log
INFO: 📁 Batch log file: notebook_logs/batch_processing/batch_processing_20250619_154702.log


✅ Batch logging configured successfully
📋 Batch log file: notebook_logs/batch_processing/batch_processing_20250619_154702.log
📊 Log level: 10 (DEBUG)
🔧 Handlers: 2 (file + console)
📂 Log directory: notebook_logs/batch_processing
📁 Individual file logs will be saved to: notebook_logs/batch_processing/individual_files
📝 This setup allows tracking both batch overview and individual file details


## 3. Initialize Batch Processor

Set up the CentroidBatchProcessor with proper configuration for BlastoSpim data processing.

In [3]:
# Load configuration for BlastoSpim data processing
config_path = Path(project_root) / "configs" / "base_config.yaml"
batch_logger.info(f"📋 Loading configuration from: {config_path}")

# Verify config file exists
if not config_path.exists():
    batch_logger.error(f"❌ Configuration file not found: {config_path}")
    raise FileNotFoundError(f"Configuration file not found: {config_path}")

# Initialize configuration manager
config_manager = ConfigManager(config_path)
config = config_manager.to_dict()

batch_logger.info("✅ Configuration loaded successfully")
batch_logger.info(f"📐 Voxel spacing: Z={config['centroid_extraction']['voxel_size']['z']:.1f}µm, "
                  f"Y/X={config['centroid_extraction']['voxel_size']['y']:.3f}µm")
batch_logger.debug(f"🔧 Full config: {config}")

# Initialize the CentroidBatchProcessor
try:
    batch_logger.info("🔄 Initializing CentroidBatchProcessor...")
    batch_processor = CentroidBatchProcessor(
        config_path=str(config_path), output_base_dir="data/labels"
    )
    batch_logger.info("✅ CentroidBatchProcessor initialized successfully")

except Exception as e:
    batch_logger.error(f"❌ Failed to initialize batch processor: {str(e)}")
    raise e

# Display initialization summary
print("✅ Batch processor ready for processing multiple files")
print(f"📋 Configuration: {config_path}")
print(f"📐 Voxel spacing: Z={config['centroid_extraction']['voxel_size']['z']:.1f}µm, Y/X={config['centroid_extraction']['voxel_size']['y']:.3f}µm")
print(f"🔧 Output will be organized in structured directories")
print(f"📊 Comprehensive logging enabled for all operations")

INFO: 📋 Loading configuration from: /mnt/home/dchhantyal/centroid_model_blastospim/configs/base_config.yaml
INFO: ✅ Configuration loaded successfully
INFO: 📐 Voxel spacing: Z=2.0µm, Y/X=0.208µm
INFO: ✅ Configuration loaded successfully
INFO: 📐 Voxel spacing: Z=2.0µm, Y/X=0.208µm
INFO: 🔄 Initializing CentroidBatchProcessor...
INFO: 🔄 Initializing CentroidBatchProcessor...
2025-06-19 15:47:11,250 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm
INFO: ✅ CentroidBatchProcessor initialized successfully
2025-06-19 15:47:11,251 - batch_processor - INFO - ✅ CentroidBatchProcessor initialized successfully
2025-06-19 15:47:11,250 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm
INFO: ✅ CentroidBatchProcessor initialized successfully
2025-06-19 15:47:11,251 - batch_processor - INFO - ✅ CentroidBatchProcessor initialized successfully


✅ Batch processor ready for processing multiple files
📋 Configuration: /mnt/home/dchhantyal/centroid_model_blastospim/configs/base_config.yaml
📐 Voxel spacing: Z=2.0µm, Y/X=0.208µm
🔧 Output will be organized in structured directories
📊 Comprehensive logging enabled for all operations


## 4. Specify Input Folder and Processing Parameters

Define the folder containing NPZ files to process and set up batch processing parameters.

### Key Parameters:
- **Input Folder**: Directory containing BlastoSpim NPZ files
- **Output Root**: Base directory for organized results
- **Processing Options**: Parallel processing, error handling, visualization settings
- **File Filtering**: Pattern matching for NPZ files

In [5]:
# Define input and output directories
input_folder = Path(project_root) / Path("data/raw/Blast") 
output_root = Path(project_root) / Path("data/labels/Blast")

batch_logger.info(f"📂 Input folder: {input_folder}")
batch_logger.info(f"📁 Output root: {output_root}")

# Create directories if they don't exist
if not input_folder.exists():
    raise FileNotFoundError(f"❌ Input folder does not exist: {input_folder}")
output_root.mkdir(parents=True, exist_ok=True)

# Search for NPZ files in the input directory
npz_pattern = str(input_folder / "*.npz")
npz_files = glob.glob(npz_pattern)
npz_files = npz_files # Limit to first 5 files for demonstration

batch_logger.info(f"🔍 Searching for NPZ files with pattern: {npz_pattern}")
batch_logger.info(f"📊 Found {len(npz_files)} NPZ files")

if len(npz_files) == 0:
    batch_logger.warning("⚠️  No NPZ files found in the specified directory")
    print("⚠️  WARNING: No NPZ files found!")
    print(f"📂 Searched in: {input_folder}")
    print(f"🔍 Pattern used: {npz_pattern}")
    print("\n💡 To proceed with this example:")
    print("   1. Place your BlastoSpim NPZ files in the 'data/raw' directory")
    print("   2. Or modify 'input_folder' path above to point to your data")
    print("   3. Re-run this cell")
else:
    print(f"✅ Found {len(npz_files)} NPZ files ready for processing")
    for i, file_path in enumerate(npz_files[:5], 1):  # Show first 5 files
        print(f"   {i}. {Path(file_path).name}")
    if len(npz_files) > 5:
        print(f"   ... and {len(npz_files) - 5} more files")

# Processing parameters
processing_params = {
    'max_workers': 8,  # Number of parallel processes (adjust based on your system)
    'save_visualizations': False,
    'save_detailed_stats': True,
    'create_summary_report': True,
    'log_individual_files': True
}

batch_logger.info(f"⚙️  Processing parameters: {processing_params}")
print(f"\n⚙️  Processing Parameters:")
for key, value in processing_params.items():
    print(f"   - {key}: {value}")

INFO: 📂 Input folder: /mnt/home/dchhantyal/centroid_model_blastospim/data/raw/Blast
2025-06-19 15:47:39,191 - batch_processor - INFO - 📂 Input folder: /mnt/home/dchhantyal/centroid_model_blastospim/data/raw/Blast
INFO: 📁 Output root: /mnt/home/dchhantyal/centroid_model_blastospim/data/labels/Blast
2025-06-19 15:47:39,192 - batch_processor - INFO - 📁 Output root: /mnt/home/dchhantyal/centroid_model_blastospim/data/labels/Blast
INFO: 🔍 Searching for NPZ files with pattern: /mnt/home/dchhantyal/centroid_model_blastospim/data/raw/Blast/*.npz
2025-06-19 15:47:39,196 - batch_processor - INFO - 🔍 Searching for NPZ files with pattern: /mnt/home/dchhantyal/centroid_model_blastospim/data/raw/Blast/*.npz
INFO: 📊 Found 80 NPZ files
2025-06-19 15:47:39,196 - batch_processor - INFO - 📊 Found 80 NPZ files
INFO: ⚙️  Processing parameters: {'max_workers': 8, 'save_visualizations': False, 'save_detailed_stats': True, 'create_summary_report': True, 'log_individual_files': True}
2025-06-19 15:47:39,197 - 

✅ Found 80 NPZ files ready for processing
   1. Blast_035.npz
   2. Blast_005.npz
   3. Blast_049.npz
   4. Blast_028.npz
   5. Blast_084.npz
   ... and 75 more files

⚙️  Processing Parameters:
   - max_workers: 8
   - save_visualizations: False
   - save_detailed_stats: True
   - create_summary_report: True
   - log_individual_files: True


## 5. Execute Batch Processing

Run the batch processor on all NPZ files with comprehensive progress tracking and error handling.

### Processing Features:
- **Progress Tracking**: Real-time progress bars and time estimates
- **Error Handling**: Continue processing even if individual files fail
- **Parallel Processing**: Process multiple files simultaneously (if enabled)
- **Detailed Logging**: Track all operations and results
- **Organized Output**: Structured directory hierarchy for results

In [6]:
# Execute batch processing - Part 1: Initialization and Setup
if len(npz_files) > 0:
    batch_logger.info(f"🚀 Starting batch processing of {len(npz_files)} files")
    batch_logger.info(f"⚙️  Processing parameters: {processing_params}")
    start_time = time.time()
    
    # Import for parallel processing
    from concurrent.futures import ThreadPoolExecutor, as_completed
    
    # Run batch processing with efficient parallel processing
    print(f"\n🚀 Processing {len(npz_files)} NPZ files...")
    print(f"📂 Input: {input_folder}")
    print(f"📁 Output: {output_root}")
    print(f"⚙️  Max workers: {processing_params['max_workers']}")
    print(f"🎨 Visualizations: {'Enabled' if processing_params['save_visualizations'] else 'Disabled'}")
    print(f"📊 Detailed stats: {'Enabled' if processing_params['save_detailed_stats'] else 'Disabled'}")
    print(f"📋 Summary report: {'Enabled' if processing_params['create_summary_report'] else 'Disabled'}")
    print(f"🔄 Parallel processing: {'Enabled' if processing_params['max_workers'] > 1 else 'Sequential'}")
    print("\n" + "="*60)
    
    # Configure batch processor based on processing_params before processing
    if not processing_params['save_visualizations']:
        # Temporarily disable visualization in config if not wanted
        original_viz_setting = batch_processor.config_manager.get("visualization.enabled", True)
        batch_processor.config_manager.set("visualization.enabled", False)
    
    print("✅ Batch processing initialized successfully!")
    print("🔧 Configuration applied based on processing parameters")
    
else:
    print("⏭️  Skipping batch processing - no NPZ files found")
    print("💡 Add NPZ files to the input directory and re-run this cell")
    print(f"🔧 Current processing parameters: {processing_params}")
    batch_results = None
    batch_processing_summary = None

INFO: 🚀 Starting batch processing of 80 files
2025-06-19 15:47:56,276 - batch_processor - INFO - 🚀 Starting batch processing of 80 files
INFO: ⚙️  Processing parameters: {'max_workers': 8, 'save_visualizations': False, 'save_detailed_stats': True, 'create_summary_report': True, 'log_individual_files': True}
2025-06-19 15:47:56,277 - batch_processor - INFO - ⚙️  Processing parameters: {'max_workers': 8, 'save_visualizations': False, 'save_detailed_stats': True, 'create_summary_report': True, 'log_individual_files': True}
2025-06-19 15:47:56,276 - batch_processor - INFO - 🚀 Starting batch processing of 80 files
INFO: ⚙️  Processing parameters: {'max_workers': 8, 'save_visualizations': False, 'save_detailed_stats': True, 'create_summary_report': True, 'log_individual_files': True}
2025-06-19 15:47:56,277 - batch_processor - INFO - ⚙️  Processing parameters: {'max_workers': 8, 'save_visualizations': False, 'save_detailed_stats': True, 'create_summary_report': True, 'log_individual_files': 


🚀 Processing 80 NPZ files...
📂 Input: /mnt/home/dchhantyal/centroid_model_blastospim/data/raw/Blast
📁 Output: /mnt/home/dchhantyal/centroid_model_blastospim/data/labels/Blast
⚙️  Max workers: 8
🎨 Visualizations: Disabled
📊 Detailed stats: Enabled
📋 Summary report: Enabled
🔄 Parallel processing: Enabled

✅ Batch processing initialized successfully!
🔧 Configuration applied based on processing parameters


In [7]:
# Execute batch processing - Part 2: Define Worker Function
if len(npz_files) > 0:
    import threading
    import matplotlib
    import matplotlib.pyplot as plt
    import warnings

    # Define the worker function for parallel processing
    def process_single_npz_file(npz_file_path):
        """Process a single NPZ file - designed for parallel execution with thread identification"""
        thread_id = threading.current_thread().ident
        worker_name = f"Worker-{thread_id}"

        try:
            # Set matplotlib backend for thread safety - CRITICAL for parallel processing
            matplotlib.use('Agg', force=True)  # Use non-interactive backend
            plt.ioff()  # Turn off interactive mode
            warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

            # Use the shared batch processor configuration but let it handle output directories automatically
            # This ensures proper visualization saving without directory conflicts
            local_processor = CentroidBatchProcessor(
                config_path=str(config_path), 
                # output_base_dir=output_root,
            )

            file_path = Path(npz_file_path)
            filename = file_path.stem


            # Use thread-safe logging with worker identification
            if 'worker_logger' in globals():
                worker_logger.info(f"🔄 [{worker_name}] Starting: {filename}")
            else:
                # Fallback to batch logger if worker logger not available
                batch_logger.info(f"🔄 Processing {filename} on {worker_name}...")

            # Process the single file with explicit visualization setting
            result = local_processor.process_single_file(
                input_file=str(file_path),
                create_visualization=processing_params["save_visualizations"],
            )

            # Force cleanup of matplotlib figures to prevent memory leaks
            plt.close('all')

            # Success logging with additional file info
            if 'worker_logger' in globals():
                worker_logger.info(f"✅ [{worker_name}] Completed: {filename}")
                if 'output_paths' in result:
                    worker_logger.info(f"📁 [{worker_name}] Output: {result['output_paths']['main_dir']}")
                    # Check if visualization was actually created
                    viz_path = result['output_paths']['visualization_dir'] / 'comprehensive_analysis.png'
                    if viz_path.exists():
                        worker_logger.info(f"🎨 [{worker_name}] Visualization saved: {viz_path}")
                    else:
                        worker_logger.warning(f"⚠️  [{worker_name}] Visualization not found: {viz_path}")
            else:
                batch_logger.info(f"✅ Completed {filename}")

            return result

        except Exception as e:
            # Clean up matplotlib on error
            plt.close('all')

            # Error logging with thread identification
            error_msg = f"❌ [{worker_name}] Failed {Path(npz_file_path).name}: {str(e)}"

            if 'worker_logger' in globals():
                worker_logger.error(error_msg)
            else:
                batch_logger.error(error_msg)

            return {
                'filename': Path(npz_file_path).stem,
                'status': 'error',
                'error': str(e),
                'worker': worker_name
            }

    print("✅ Worker function defined with improved visualization handling")
    print("🔧 Each worker will create its own processor instance with proper configuration")
    print("📊 Thread identification enabled for better progress tracking")
    print("🎨 Visualization saving fixed - using config defaults instead of custom output directory")
    print("🖼️  Matplotlib backend set to 'Agg' for thread-safe visualization")
    print("🧹 Added automatic cleanup of matplotlib figures to prevent memory leaks")

else:
    print("⏭️  Skipping worker function definition - no NPZ files found")

✅ Worker function defined with improved visualization handling
🔧 Each worker will create its own processor instance with proper configuration
📊 Thread identification enabled for better progress tracking
🎨 Visualization saving fixed - using config defaults instead of custom output directory
🖼️  Matplotlib backend set to 'Agg' for thread-safe visualization
🧹 Added automatic cleanup of matplotlib figures to prevent memory leaks


In [8]:
# Execute batch processing - Part 3: Run Parallel or Sequential Processing
if len(npz_files) > 0:
    import threading
    import sys
    from io import StringIO
    
    # Create a thread-safe logger for worker processes
    worker_logger = logging.getLogger('worker_processor')
    worker_logger.setLevel(logging.INFO)
    
    # Clear any existing handlers for worker logger
    worker_logger.handlers.clear()
    
    # Create a custom formatter for worker logs with thread identification
    worker_formatter = logging.Formatter(
        '%(asctime)s - [Worker-%(thread)d] - %(levelname)s - %(message)s',
        datefmt='%H:%M:%S'
    )
    
    # Console handler for worker progress with minimal format
    worker_console_handler = logging.StreamHandler(sys.stdout)
    worker_console_handler.setLevel(logging.INFO)
    worker_console_handler.setFormatter(worker_formatter)
    worker_logger.addHandler(worker_console_handler)
    
    try:
        # Execute parallel processing if max_workers > 1, otherwise sequential
        results = []
        processing_start_time = time.time()
        
        if processing_params['max_workers'] > 1:
            print(f"🔄 Starting parallel processing with {processing_params['max_workers']} workers...")
            print(f"📊 Progress tracking with thread identification enabled")
            print("-" * 80)
            
            # Use ThreadPoolExecutor for I/O-bound operations (better for file processing)
            # ProcessPoolExecutor can be used for CPU-intensive tasks, but ThreadPoolExecutor
            # is often better for file I/O and works well with our processing pipeline
            with ThreadPoolExecutor(max_workers=processing_params['max_workers']) as executor:
                # Submit all files for processing
                future_to_file = {
                    executor.submit(process_single_npz_file, npz_file): npz_file 
                    for npz_file in npz_files
                }
                
                # Collect results as they complete
                completed_count = 0
                for future in as_completed(future_to_file):
                    npz_file = future_to_file[future]
                    try:
                        result = future.result()
                        results.append(result)
                        completed_count += 1
                        
                        # Progress reporting with cleaner formatting
                        progress_percent = (completed_count / len(npz_files)) * 100
                        elapsed_time = time.time() - processing_start_time
                        avg_time_per_file = elapsed_time / completed_count
                        estimated_remaining = avg_time_per_file * (len(npz_files) - completed_count)
                        
                        # Clean progress output with consistent formatting
                        filename_display = Path(npz_file).name[:30] + "..." if len(Path(npz_file).name) > 30 else Path(npz_file).name
                        status_icon = "✅" if result.get('status') == 'success' else "❌"
                        
                        print(f"{status_icon} [{completed_count:2d}/{len(npz_files)}] ({progress_percent:5.1f}%) "
                              f"| ETA: {estimated_remaining:5.1f}s | {filename_display}")
                        
                    except Exception as e:
                        batch_logger.error(f"❌ Worker failed for {Path(npz_file).name}: {str(e)}")
                        results.append({
                            'filename': Path(npz_file).stem,
                            'status': 'error',
                            'error': str(e)
                        })
                        completed_count += 1
                        
                        # Error progress display
                        progress_percent = (completed_count / len(npz_files)) * 100
                        filename_display = Path(npz_file).name[:30] + "..." if len(Path(npz_file).name) > 30 else Path(npz_file).name
                        print(f"❌ [{completed_count:2d}/{len(npz_files)}] ({progress_percent:5.1f}%) "
                              f"| ERROR     | {filename_display}")
            
            print("-" * 80)
            print(f"✅ Parallel processing completed with {processing_params['max_workers']} workers!")
            
        else:
            print(f"🔄 Starting sequential processing...")
            print("-" * 80)
            
            # Sequential processing with better progress display
            for i, npz_file in enumerate(npz_files, 1):
                filename_display = Path(npz_file).name[:30] + "..." if len(Path(npz_file).name) > 30 else Path(npz_file).name
                print(f"🔄 [{i:2d}/{len(npz_files)}] Processing: {filename_display}")
                
                result = process_single_npz_file(npz_file)
                results.append(result)
                
                # Status update
                status_icon = "✅" if result.get('status') == 'success' else "❌"
                print(f"{status_icon} [{i:2d}/{len(npz_files)}] Completed: {filename_display}")
            
            print("-" * 80)
        
        # Restore original visualization setting if changed
        if not processing_params['save_visualizations']:
            batch_processor.config_manager.set("visualization.enabled", original_viz_setting)
        
        # Clean up worker logger
        worker_logger.handlers.clear()
        
        print("✅ Core processing completed successfully!")
        print(f"📊 Processed {len(results)} files with clean output formatting")
        
    except Exception as e:
        # Clean up worker logger on error
        worker_logger.handlers.clear()
        
        batch_logger.error(f"❌ Batch processing failed: {str(e)}")
        print(f"\n❌ ERROR during batch processing: {str(e)}")
        print(f"💡 Check processing parameters: {processing_params}")
        raise e
        
else:
    print("⏭️  Skipping core processing - no NPZ files found")
    results = []

2025-06-19 15:48:20,853 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm


🔄 Starting parallel processing with 8 workers...
📊 Progress tracking with thread identification enabled
--------------------------------------------------------------------------------
15:48:20 - [Worker-23454127777536] - INFO - 🔄 [Worker-23454127777536] Starting: Blast_035


2025-06-19 15:48:20,860 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm
2025-06-19 15:48:20,861 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm
2025-06-19 15:48:20,865 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm
2025-06-19 15:48:20,861 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm
2025-06-19 15:48:20,865 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm


15:48:20 - [Worker-23454147065600] - INFO - 🔄 [Worker-23454147065600] Starting: Blast_005


2025-06-19 15:48:20,866 - src.preprocessing.centroid_batch_processor - INFO - Using voxel spacing from config: Z=2.0µm, Y=0.208µm, X=0.208µm


...

15:50:18 - [Worker-23454131980032] - INFO - 📁 [Worker-23454131980032] Output: /mnt/home/dchhantyal/centroid_model_blastospim/data/labels/Blast/label_Blast_027


2025-06-19 15:50:18,270 - worker_processor - INFO - 📁 [Worker-23454131980032] Output: /mnt/home/dchhantyal/centroid_model_blastospim/data/labels/Blast/label_Blast_027


15:50:18 - [Worker-23454131980032] - WARNING - ⚠️  [Worker-23454131980032] Visualization not found: /mnt/home/dchhantyal/centroid_model_blastospim/data/labels/Blast/label_Blast_027/visualizations/comprehensive_analysis.png


2025-06-19 15:50:18,271 - worker_processor - WARNING - ⚠️  [Worker-23454131980032] Visualization not found: /mnt/home/dchhantyal/centroid_model_blastospim/data/labels/Blast/label_Blast_027/visualizations/comprehensive_analysis.png


✅ [80/80] (100.0%) | ETA:   0.0s | Blast_027.npz
--------------------------------------------------------------------------------
✅ Parallel processing completed with 8 workers!
✅ Core processing completed successfully!
📊 Processed 80 files with clean output formatting


## 6. Analyze Batch Processing Results

Examine the results from batch processing, including success rates, processing statistics, and any errors encountered.

### Analysis Features:
- **Success/Failure Summary**: Overall batch processing statistics
- **Processing Time Analysis**: Time distribution across files
- **Error Analysis**: Detailed examination of any failed files
- **Output Organization**: Review of generated file structure
- **Quality Metrics**: Distribution of extracted centroids and volumes

## 7. Explore Individual Results

Examine specific results from individual files, including generated visualizations and detailed statistics.

### Exploration Options:
- **File Selection**: Choose specific files to examine in detail
- **Visualization Review**: Display generated plots and analysis
- **Statistics Deep Dive**: Detailed metrics for individual samples
- **Output File Inspection**: Examine saved JSON results and logs

## 8. Summary and Next Steps

### Batch Processing Complete! 🎉

This notebook has successfully demonstrated batch processing of BlastoSpim NPZ files using the **CentroidBatchProcessor** with the following achievements:

#### ✅ **What Was Accomplished:**
- **Comprehensive Batch Processing**: Processed multiple NPZ files in parallel
- **Robust Error Handling**: Continued processing even when individual files failed
- **Detailed Logging**: Tracked all operations with timestamps and progress
- **Organized Output Structure**: Created structured directories for each processed file
- **Physical Coordinate Calculations**: Used correct voxel spacing (Z=2.0µm, Y/X=0.208µm)
- **Professional Visualizations**: Generated multi-slice, 3D, and statistical plots
- **Comprehensive Statistics**: Calculated centroids, volumes, and bounding boxes
- **Batch Analysis**: Summarized results across all processed files

#### 📊 **Generated Outputs:**
- **Individual Results**: Separate directory for each processed file
- **Visualizations**: PNG files with comprehensive plots
- **Statistics**: JSON files with detailed metrics
- **Summary Reports**: Batch-level analysis and statistics
- **Detailed Logs**: Complete processing history with timestamps

#### 🔧 **Key Features Used:**
- **CentroidBatchProcessor**: Main batch processing engine
- **ConfigManager**: YAML-based configuration management
- **Progress Tracking**: Real-time progress bars and time estimates
- **Parallel Processing**: Multi-threaded processing for efficiency
- **Physical Units**: Proper handling of anisotropic voxel spacing

#### 🚀 **Next Steps:**
1. **Review Generated Results**: Examine the output directories and visualizations
2. **Analyze Statistics**: Use the batch summary for scientific analysis
3. **Customize Processing**: Modify configuration parameters as needed
4. **Scale Up**: Process larger datasets with the same pipeline
5. **Integration**: Incorporate results into downstream analysis workflows

#### 💡 **Tips for Production Use:**
- Adjust `max_workers` based on your system's capabilities
- Monitor memory usage for large datasets
- Use the logging files to troubleshoot any processing issues
- Customize visualization parameters in the configuration file
- Consider implementing additional quality control checks

**The batch processing pipeline is now ready for production use with your BlastoSpim data!**